<img src="https://cdn.pixabay.com/photo/2016/11/30/12/17/cells-1872666_960_720.jpg" width="600">
<i> Image by <a href="https://pixabay.com/users/qimono-1962238/?utm_source=link-attribution&amp;utm_medium=referral&amp;utm_campaign=image&amp;utm_content=1872666">Arek Socha</a> from <a href="https://pixabay.com/?utm_source=link-attribution&amp;utm_medium=referral&amp;utm_campaign=image&amp;utm_content=1872666">Pixabay</a> </i>

# <font color='blue'> Introduction  </font>

There has been a change in drug discovery procedure in the last years,to a more targeted model based on the understanding of the underlying biological mechanism of a disease. Scientists seek to identify a protein target associated with a disease and develop a molecule that can modulate that protein target. Mechanism-of-action (MoA) is a label that describes the biological activity of a given molecule. One approach is to treat a sample of human cells with the drug and then analyze the cellular responses with algorithms that search for similarity to known patterns in large genomic databases, such as libraries of gene expression or cell viability patterns of drugs with known MoAs. For this competition, we will be predicting the probability that the sample(sig_id) had a positive response for each MoA target , given various inputs such as gene expression data and cell viability data. The data is based on a new technology that measures simultaneously (within the same samples) human cells’ responses to drugs in a pool of 100 different cell types (thus solving the problem of identifying ex-ante, which cell types are better suited for a given drug). Drugs can have multiple MoA annotations, so the task is formally a multi-label classification problem. This kernel has been influenced by great kernels posted by other fellow kagglers and is my first try in a research contest. Thank you all for sharing all this great content!

<a id="s1"></a>
### <font color='blue'> Importing libraries </font>

In [ ]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from sklearn import preprocessing                            
from sklearn.decomposition import PCA, FastICA

import tensorflow as tf

from tensorflow import keras
from keras.models import Sequential 
from keras import models, utils,backend
import keras.utils 
from keras.optimizers import Adam   
from keras import layers 
from keras.layers import Activation, Dense ,Dropout, BatchNormalization, Input,LeakyReLU
from keras.utils import np_utils
from keras.callbacks import EarlyStopping ,ModelCheckpoint,ReduceLROnPlateau
from sklearn.model_selection import StratifiedKFold
from keras.models import model_from_json  


import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

### <font color='blue'> Loading datasets </font>

In [ ]:
trainf = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
testf = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
sub = pd.read_csv('../input/lish-moa/sample_submission.csv')
targets = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
targets1 = pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv')

train = trainf[trainf.cp_type != 'ctl_vehicle'].drop(columns=['cp_type'])
test = testf[testf.cp_type != 'ctl_vehicle'].drop(columns=['cp_type'])
Atrain = train
Atest = test

<a id="s3"></a>
## <font color='blue'> Exploring the data</font>

We will load two data sets, train and test. Train data set is used for model training and test data set will be the unseen data used to make predictions, lets see now how they look like.

In [ ]:
train.head()

In [ ]:
test.head()

### <font color='blue'>     Missing Values</font>

In [ ]:
# Lets see size and check for Nulls
print('Train dataset',train.shape)
print('Test dataset',test.shape)
missing_train=(train.isnull().sum()).sum()
missing_test=(test.isnull().sum()).sum()
print('Missing values in train set:',missing_train,'Missing values in test set:',missing_test)

In [ ]:
categ_feat_train = train.select_dtypes(include=['object'])
num_feat_train = train.select_dtypes(exclude=['object'])
print('Numerical features',len(num_feat_train.columns))
print('Categorical features',len(categ_feat_train.columns))
print(categ_feat_train.columns)

So, there are 876 columns in train and test sets, three of them categorical, and there are no missing values. 

* sig_id is the unique primary key of the sample 
* Features with g- : are gene expression levels and there are 772 of them (from g-0 to g-771) 
* Features with c- : are cell viability measurements for each cell line, there are 100 of them (from c-0 to g-99) 
* Features with cp_:  
  cp_type: samples are treated with a compound(trt_cp) or with a control perturbation (ctl_vehicle) that has no MoAs.   
  cp_time: duration of the treatment (24, 48 or 72 hours)  
  cp_dose: dosage of the treatment low/high (D1/D2)


### <font color='blue'>  Cp_type, cp_time and cp_dose features </font>
There are three categorical features cp_type,cp_time and cp_dose. Let us see how their values are distributed in the training dataset. 

In [ ]:
matplotlib.rc('figure', figsize=(20, 12))
fig, ax =plt.subplots(2,3)
sns.countplot(x='cp_type', data=trainf,ax=ax[0,0])
ax[0,0].set_title('TRAIN- Compound / control treatment', fontsize=14, weight='bold')
sns.countplot(x='cp_dose', data=trainf,ax=ax[0,1])
ax[0,1].set_title('TRAIN- Doses  Low / High', fontsize=14,weight='bold')
sns.countplot(x='cp_time', data=trainf,ax=ax[0,2])
ax[0,2].set_title('TRAIN- Treatment duration (hours)', fontsize=14,weight='bold')
sns.countplot(x='cp_type', data=testf,ax=ax[1,0])
ax[1,0].set_title('TEST- Compound / control treatment', fontsize=14, weight='bold')
sns.countplot(x='cp_dose', data=testf,ax=ax[1,1])
ax[1,1].set_title('TEST- Doses  Low / High', fontsize=14,weight='bold')
sns.countplot(x='cp_time', data=testf,ax=ax[1,2])
ax[1,2].set_title('TEST- Treatment duration (hours)', fontsize=14,weight='bold')
plt.show()

As we can see doses and treatment duration times are distributed equally but only 8% of the samples are treated with a control perturbation. Distributions are similar both in training and test sets ,probably indicating that they follow same experiment setup (since each combo of drug-timing-dose can be viewed as one independent experiment, so there are 6 independent realizations per drug, although some drugs have been profiled more than once). 

## <font color='blue'> Gene features  </font>
 
The role of genes is to encode proteins who dictate how a cell functions. So, genes expressed in a particular cell determine what that cell can do. We can see that genes expression values in our data sets show normal like distribution, with zero mean as random following plots show.

In [ ]:
matplotlib.rc('figure', figsize=(20, 4))
fig, ax =plt.subplots(1,4)
fig.suptitle('Genes distributions', fontsize=16)
sel_genes = [7,16,33,66]
i=0
for item in sel_genes:
    train.hist(column=['g-'+ str(item)], ax=ax[i])
    i+=1
plt.show()

In [ ]:
matplotlib.rc('figure', figsize=(20, 4))
fig.suptitle('Genes distributions G-0 to G-771', fontsize=16)
sel_genes = list(range(0, 771))
for item in sel_genes:
    sns.kdeplot(data=train['g-'+ str(item)], shade=False,legend=False)
plt.show()

## <font color='blue'>Genes correlation analysis </font>

It would probably be useful to examine if there is any pairwise correlation (suggesting  a biological relationship) between genes, such that changes in the expression levels of one gene correspond to changes in the expression level of another gene.
There are references though like [this ](https://www.frontiersin.org/articles/10.3389/fmicb.2015.00650/full) , suggesting that “ …. computation of pairwise gene associations (correlation; mutual information) produces unexpectedly large variation in estimates of pairwise gene association—regardless of the metric used, the organism under study, or the number and source of the samples probably due to sampling bias.” And also “….many individual genes show small differences in absolute gene expression levels across the set of samples. These small differences are due mainly to “noise” instead of “signal” attributable to environmental or genetic perturbations. 
So we will not examine corellations but we can keep that ICA as we shall see later could be used as a helpful denoising step even though it might be sightly biased towards highly expressed genes.

## <font color='blue'> Cell features analysis </font>

These features are related to cell viability, high negative cell viability values reflect a high numbers of cell deaths and low values high survival rates. Their distributions are skewed with a normal-like look but with heavy negative tails and peaks at -10 probably due to experiment data normalization procedure. 

In [ ]:
matplotlib.rc('figure', figsize=(20, 4))
fig, ax =plt.subplots(1,4)
fig.suptitle('Cells distributions', fontsize=16)
sel_cells = [7,16,30,45]
i=0
for item in sel_cells:
    train.hist(column=['c-'+ str(item)], ax=ax[i])
    i+=1
plt.show()

In [ ]:
matplotlib.rc('figure', figsize=(20, 4))
fig.suptitle('Cells distributions c-0 to c-771', fontsize=16)
sel_cells = list(range(0, 99))
for item in sel_cells:
    sns.kdeplot(data=train['c-'+ str(item)], shade=False,legend=False)
plt.show()

### <font color='blue'> Cell correlation analysis </font>
Let's explore how cells are correlated.

In [ ]:
Cells = [c for c in train.columns if "c-" in c]
plt.figure(figsize=(10,6))
sns.heatmap(train[Cells].corr(), cmap='viridis')
plt.title('Cell viability correlations (Train set)', fontsize=14, weight='bold')
plt.show()

There is a clear high correlation between cell viabilities that has to be examined. 

<a id="s4"></a>
## <font color='blue'> Targets </font>

There are two target files containing MOAs of interest, the first one (train_targets_scored.csv) contains features that are scored and an auxiliary one (train_targets_nonscored.csv) having elements that are not scored . Let's see what type of features they have

In [ ]:
targets.head()

In [ ]:
targets.columns

The targets we have to predict are basically probabilities of activation for each of various proteins-targets. This is a multi-label problem since one sample can be classified to multiple targets or none. The main target types as we can see are activators, inhibitors, receptors agonists and antagonists, agents, stimulants. 

* Receptors are chemical structures, composed of protein, that receive and transduce signals that may be integrated into biological systems.[1] These signals are typically chemical messengers which bind to a receptor and cause some form of cellular/tissue response, e.g. a change in the electrical activity of a cell.
 * Agonists are chemicals that bind to a receptors and activate them to produce a biological response.  
 * An antagonist blocks the action of the agonist, while an inverse agonist causes an action opposite to that of the agonist.
* Activators : They are proteins that increase transcription of a gene or set of genes. Activators are considered to have positive control over gene expression, as they function to promote gene transcription and, in some cases, are required for the transcription of genes to occur. Most activators are DNA-binding proteins that bind to enhancers  
* Inhibitors  : An enzyme inhibitor is a molecule that binds to an enzyme and decreases its activity
   
  



In [ ]:
target_classes1 = targets.drop(['sig_id'], axis=1).astype(bool).sum(axis=1).reset_index()
target_classes1.columns = ['Sig_Ids', 'activations']
target_classes1 = target_classes1.groupby(['activations'])['Sig_Ids'].count().reset_index()
target_classes2 = targets1.drop(['sig_id'], axis=1).astype(bool).sum(axis=1).reset_index()
target_classes2.columns = ['Sig_Ids', 'activations']
target_classes2 = target_classes2.groupby(['activations'])['Sig_Ids'].count().reset_index()
matplotlib.rc('figure', figsize=(8, 5))
fig, ax =plt.subplots(1,2)
sns.barplot(x="activations", y="Sig_Ids", data=target_classes1, ax=ax[0]).set_title('Scored targets')
sns.barplot(x="activations", y="Sig_Ids", data=target_classes2, ax=ax[1]).set_title('NonScored targets')
plt.show()

We can see that numbers of activations differ for scored and non_scored datasets, but in any case more than 90% of the drugs activate zero or one of the target columns.

## <font color='blue'> Removing uninformative features </font>

The fewer and more useful features we have, the better for our models. An interesting idea could be to try to find and remove any "uninformative" genes in our data sets like those ones with very low expression values in most samples or those whose expression values shows small variation throughout samples. Removing those feature could help our models perform better and faster. We have to remember though that what we see comes after z-score and data quantile normalization so we have to do some "reverse enginineering", but there is no way to get back to values before transformations since statistics like mean and std are lost. So we will use data as is and try VarianceThreshold to detect and remove low variance features (we have already scaled our data). Target variability was set to 90% with trial and error. 

In [ ]:
#For Model1

combo = pd.concat([train, test], axis=0)
cols=combo.columns.tolist()
catego = combo.iloc[:, 0:3]
genes_df =combo[cols[3:775]]
cells_df = combo[cols[775:]]

In [ ]:
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
thr = VarianceThreshold(0.99)  

# Model1
VT_ALL = thr.fit_transform(combo[cols[3:]])
# Model2
cells_df=thr.fit_transform(cells_df)
genes_df=thr.fit_transform(genes_df)


## <font color='blue'>  Quantile Transformation </font>

It might be helpful for our models to change distributions using QuantileTransformer, that provides non-linear transformations in which distances between marginal outliers and inliers are shrunk. StandardScaler and MinMaxScaler were also tested but since they are very sensitive to outliers, I didn't use them.

In [ ]:
from sklearn.preprocessing import QuantileTransformer

QUA = QuantileTransformer(n_quantiles=250, output_distribution='normal')

#Model1
VT_ALL = QUA.fit_transform(VT_ALL)
#Model2
cells_df = QUA.fit_transform(cells_df)
genes_df = QUA.fit_transform(genes_df)

# <font color='blue'> Dimensionality reduction </font>

## Using ICA to reduce dimensionality

I examined both PCA and ICA to perform an initial reduction in the dimensionality of the input dataset while still preserving most of the important data structure. There is work presenting the use of ICA in dimensionality reduction, deconvolution, data pre-processing, meta-analysis, and others applied to different data types (transcriptome, methylome, proteome, single-cell data) like [this one](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6771121/) and also references that ICA can be selected over PCA depending on the case. My best results were with ICA so I will use it in this kernel. Determining the optimal number of independent components is a difficult task and is selected by trials, [this article ](https://bmcgenomics.biomedcentral.com/articles/10.1186/s12864-017-4112-9) can be very informative.
<img src="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6771121/bin/ijms-20-04414-g001.jpg" width="600">


In [ ]:
#### Model1
all_ica = FastICA(n_components=500,max_iter=500)
VT_ALL_ica=all_ica.fit_transform(VT_ALL)

VT_ALL_ica_df = pd.DataFrame(VT_ALL_ica , columns=["VTICA" + str(i) for i in range(500)], index=combo.index)
full= combo.iloc[:, 1:]

new_combo= pd.concat([full,VT_ALL_ica_df], axis=1 )
train = new_combo[ : train.shape[0]]
test = new_combo[-test.shape[0] : ]

In [ ]:
##########################  model 2 ##########################

c_ica = FastICA(n_components=50,max_iter=500)
cells_ica=c_ica.fit_transform(cells_df)
g_ica = FastICA(n_components=300,max_iter=500)
genes_ica=g_ica.fit_transform(genes_df)

cells_ica_df = pd.DataFrame(cells_ica , columns=["C-ICA" + str(i) for i in range(50)], index=combo.index)
genes_ica_df = pd.DataFrame(genes_ica , columns=["G-ICA" + str(i) for i in range(300)], index=combo.index)

full= combo.iloc[:, 1:]
new_combo2= pd.concat([full,cells_ica_df,genes_ica_df], axis=1 )
train2 = new_combo2[ : train.shape[0]]
test2 = new_combo2[-test.shape[0] : ]

# <font color='blue'>  Categorical Data </font>
 In the following part we shall convert all categorical features into dummy/indicator variables

In [ ]:
train['cp_time'] = train['cp_time'].map( {24: 1, 48: 2, 72: 3} ).astype(int)
#train = pd.get_dummies(train, columns = ["cp_type"], prefix="CPTP",drop_first=True)
train = pd.get_dummies(train, columns = ["cp_dose"], prefix="CPD", drop_first=True)

test['cp_time'] = test['cp_time'].map( {24: 1, 48: 2, 72: 3} ).astype(int)
#test = pd.get_dummies(test, columns = ["cp_type"], prefix="CPTP",drop_first=True)
test = pd.get_dummies(test, columns = ["cp_dose"], prefix="CPD",drop_first=True)

In [ ]:
train2['cp_time'] = train2['cp_time'].map( {24: 1, 48: 2, 72: 3} ).astype(int)
#train = pd.get_dummies(train, columns = ["cp_type"], prefix="CPTP",drop_first=True)
train2 = pd.get_dummies(train2, columns = ["cp_dose"], prefix="CPD", drop_first=True)

test2['cp_time'] = test2['cp_time'].map( {24: 1, 48: 2, 72: 3} ).astype(int)
#test = pd.get_dummies(test, columns = ["cp_type"], prefix="CPTP",drop_first=True)
test2 = pd.get_dummies(test2, columns = ["cp_dose"], prefix="CPD",drop_first=True)

In [ ]:
targets0 = targets.loc[Atrain.index]
y = targets0.drop(['sig_id'], axis=1)

X = train.values
Xtest = test.values

#Model2
X2 = train2.values
Xtest2 = test2.values

## <font color='blue'> Multi-label Models </font>

In [ ]:
def build_model():
    keras.backend.clear_session()
    model = models.Sequential()
    n_cols = X.shape[1]
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(2048,activation='relu', input_shape=(n_cols,)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(2048,activation='relu' ))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(206, activation='sigmoid'))
    return model   

In [ ]:
## Model1  Smoothing 0.0001
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_logloss', factor=0.3, patience=5, mode='min', min_lr=1E-5)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_logloss', min_delta=1E-5, patience=15, mode='min',restore_best_weights=True)

n_labels = y.shape[1]
n_features = X.shape[1]
n_train = X.shape[0]
n_test = Xtest.shape[0]


# Label smoothing

# Since our score is heavily punished by very confident (very close to 0/1) incorrect answers
#as @Rahul suggests, labels will be smoothed to a small extent, and predictions are clipped. There
# is a build in option for label smoothing in BinaryCrossentropy set to a small value

def logloss(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred,0.0001,0.9999)
    return -backend.mean(y_true*backend.log(y_pred) + (1-y_true)*backend.log(1-y_pred))

n_seeds = 6
np.random.seed(1)
seeds = np.random.randint(0,100,size=n_seeds)

n_folds = 5
y_pred1 = np.zeros((n_test,n_labels))
oof = tf.constant(0.0)
hists = []
for seed in seeds:
    fold = 0
    mltsplit = MultilabelStratifiedKFold(n_splits=n_folds,shuffle=True,random_state=seed)
    for train, test in mltsplit.split(X,y.values):
        X_train = X[train]
        X_test  = X[test]
        y_train = y.values[train]
        y_test  = y.values[test]

        model=build_model()
        model.compile(optimizer=keras.optimizers.Adam(lr=1e-3, decay=1e-3 / 200), loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.0001), metrics=logloss)

        hist = model.fit(X_train,y_train, batch_size=128, epochs=50,verbose=0,validation_data = (X_test,y_test),callbacks=[reduce_lr, early_stopping])
        hists.append(hist)
        model.save('LabelSmoothed_seed_'+str(seed)+'_fold_'+str(fold))
        y_val = model.predict(X_test)
        oof += logloss(tf.constant(y_test,dtype=tf.float32),tf.constant(y_val,dtype=tf.float32))/(n_folds*n_seeds)
        y_pred1 += model.predict(Xtest)/(n_folds*n_seeds)
        print('seed=',seed,'fold=',fold)
        fold += 1
print('OOF score is ',oof)

plt.figure(figsize=(12,8))
hist_train = np.zeros(40)
hist_val = np.zeros(40)
for i in range(n_folds):
    hist_train += np.array(hists[i].history['loss'][:40])/n_folds
    hist_val += np.array(hists[i].history['val_loss'][:40])/n_folds

plt.plot(hist_train)
plt.plot(hist_val)
plt.yscale('log')
plt.yticks(ticks=[1,1E-1,1E-2])
plt.xlabel('Epochs')
plt.ylabel('Average Logloss')
plt.legend(['Training','Validation'])

In [ ]:
## Model2  No smoothing
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=10, mode='min', min_lr=1E-5)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1E-5, patience=15, mode='min',restore_best_weights=True)

n_labels = y.shape[1]
n_features = X.shape[1]
n_train = X.shape[0]
n_test = Xtest.shape[0]

n_seeds = 6
np.random.seed(1)
seeds = np.random.randint(0,100,size=n_seeds)

n_folds = 5
y_pred2 = np.zeros((n_test,n_labels))
oof = tf.constant(0.0)
hists = []
for seed in seeds:
    fold = 0
    mltsplit = MultilabelStratifiedKFold(n_splits=n_folds,shuffle=True,random_state=seed)
    for train, test in mltsplit.split(X,y.values):
        X_train = X[train]
        X_test  = X[test]
        y_train = y.values[train]
        y_test  = y.values[test]

        model=build_model()
        model.compile(optimizer=keras.optimizers.Adam(lr=1e-3, decay=1e-3 / 200), loss=tf.keras.losses.BinaryCrossentropy())

        hist = model.fit(X_train,y_train, batch_size=128, epochs=50,verbose=0,validation_data = (X_test,y_test),callbacks=[reduce_lr, early_stopping])
        hists.append(hist)
        model.save('LabelSmoothed_seed_'+str(seed)+'_fold_'+str(fold))
        y_val = model.predict(X_test)
        oof += logloss(tf.constant(y_test,dtype=tf.float32),tf.constant(y_val,dtype=tf.float32))/(n_folds*n_seeds)
        y_pred2 += model.predict(Xtest)/(n_folds*n_seeds)
        print('seed=',seed,'fold=',fold)
        fold += 1
print('OOF score is ',oof)

plt.figure(figsize=(12,8))
hist_train = np.zeros(40)
hist_val = np.zeros(40)
for i in range(n_folds):
    hist_train += np.array(hists[i].history['loss'][:40])/n_folds
    hist_val += np.array(hists[i].history['val_loss'][:40])/n_folds

plt.plot(hist_train)
plt.plot(hist_val)
plt.yscale('log')
plt.yticks(ticks=[1,1E-1,1E-2])
plt.xlabel('Epochs')
plt.ylabel('Average Logloss')
plt.legend(['Training','Validation'])

In [ ]:
# model TYPE2 ICA 50 ICA 300
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_logloss', factor=0.3, patience=5, mode='min', min_lr=1E-5)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_logloss', min_delta=1E-5, patience=15, mode='min',restore_best_weights=True)

n_labels = y.shape[1]
n_features = X2.shape[1]
n_train = X2.shape[0]
n_test = Xtest2.shape[0]

n_seeds = 6
np.random.seed(1)
seeds = np.random.randint(0,100,size=n_seeds)

n_folds = 5
y_pred3 = np.zeros((n_test,n_labels))
oof = tf.constant(0.0)
hists = []
for seed in seeds:
    fold = 0
    mltsplit = MultilabelStratifiedKFold(n_splits=n_folds,shuffle=True,random_state=seed)
    for train, test in mltsplit.split(X2,y.values):
        X_train = X2[train]
        X_test  = X2[test]
        y_train = y.values[train]
        y_test  = y.values[test]

        model=build_model()
        model.compile(optimizer=keras.optimizers.Adam(lr=1e-3, decay=1e-3 / 200), loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.001), metrics=logloss)

        hist = model.fit(X_train,y_train, batch_size=128, epochs=50,verbose=0,validation_data = (X_test,y_test),callbacks=[reduce_lr, early_stopping])
        hists.append(hist)
        model.save('LabelSmoothed_seed_'+str(seed)+'_fold_'+str(fold))
        y_val = model.predict(X_test)
        oof += logloss(tf.constant(y_test,dtype=tf.float32),tf.constant(y_val,dtype=tf.float32))/(n_folds*n_seeds)
        y_pred3 += model.predict(Xtest2)/(n_folds*n_seeds)
        print('seed=',seed,'fold=',fold)
        fold += 1
print('OOF score is ',oof)

plt.figure(figsize=(12,8))
hist_train = np.zeros(40)
hist_val = np.zeros(40)
for i in range(n_folds):
    hist_train += np.array(hists[i].history['loss'][:40])/n_folds
    hist_val += np.array(hists[i].history['val_loss'][:40])/n_folds

plt.plot(hist_train)
plt.plot(hist_val)
plt.yscale('log')
plt.yticks(ticks=[1,1E-1,1E-2])
plt.xlabel('Epochs')
plt.ylabel('Average Logloss')
plt.legend(['Training','Validation'])

In [ ]:
## Model4  TYPE1 No smoothing
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=10, mode='min', min_lr=1E-5)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1E-5, patience=15, mode='min',restore_best_weights=True)

n_labels = y.shape[1]
n_features = X.shape[1]
n_train = X.shape[0]
n_test = Xtest.shape[0]

n_seeds = 6
np.random.seed(1)
seeds = np.random.randint(0,100,size=n_seeds)

n_folds = 5
y_pred4 = np.zeros((n_test,n_labels))
oof = tf.constant(0.0)
hists = []
for seed in seeds:
    fold = 0
    mltsplit = MultilabelStratifiedKFold(n_splits=n_folds,shuffle=True,random_state=seed)
    for train, test in mltsplit.split(X,y.values):
        X_train = X[train]
        X_test  = X[test]
        y_train = y.values[train]
        y_test  = y.values[test]

        model=build_model()
        model.compile(optimizer=keras.optimizers.Adam(lr=1e-3, decay=1e-3 / 200), loss=tf.keras.losses.BinaryCrossentropy())

        hist = model.fit(X_train,y_train, batch_size=128, epochs=50,verbose=0,validation_data = (X_test,y_test),callbacks=[reduce_lr, early_stopping])
        hists.append(hist)
        model.save('LabelSmoothed_seed_'+str(seed)+'_fold_'+str(fold))
        y_val = model.predict(X_test)
        oof += logloss(tf.constant(y_test,dtype=tf.float32),tf.constant(y_val,dtype=tf.float32))/(n_folds*n_seeds)
        y_pred4 += model.predict(Xtest)/(n_folds*n_seeds)
        print('seed=',seed,'fold=',fold)
        fold += 1
print('OOF score is ',oof)

plt.figure(figsize=(12,8))
hist_train = np.zeros(40)
hist_val = np.zeros(40)
for i in range(n_folds):
    hist_train += np.array(hists[i].history['loss'][:40])/n_folds
    hist_val += np.array(hists[i].history['val_loss'][:40])/n_folds

plt.plot(hist_train)
plt.plot(hist_val)
plt.yscale('log')
plt.yticks(ticks=[1,1E-1,1E-2])
plt.xlabel('Epochs')
plt.ylabel('Average Logloss')
plt.legend(['Training','Validation'])

## <font color='blue'> Experimenting with model parameters and pre/post processing data </font>

I tested various techniques and parameters during this work, some of them improved my LB significally.  

1) Using cp_type column : predictions after removing this feature were less accurate.  
2) Using Standard /Minmax scaling but Normal Quantile Transform before ICA lead to better predictions  
3) Label smoothing is definitely leading to improved scores due to the way of LB scoring system. Trying to use values larger than 0.001/0.999 for min/max lowered scores.  
4) Adding weight normalization was tested too but without any success  
5) Testing AdamW, LazyAdam optimizers, didn't improve results  
6) Testing Leaky_relu ,elu no obvious improvement  
7) Changing batch size from 128 up to 512 didn't change LB score (as expected)  
8) Using larger models : more hidden layers and/or nodes per layer up to 8192/2048 showed small improvement (possibly due to some extra overfitting)  
9) Tried batch normalization before the activation function, not suitable for our activations  
10) Tested different values for FastICA independent components

In [ ]:
pred_df = pd.DataFrame(y_pred, index =[Atest.index]) 
pred_df2 = pd.DataFrame(y_pred2, index =[Atest.index]) 
pred_df3 = pd.DataFrame(y_pred3, index =[Atest.index]) 
pred_df4 = pd.DataFrame(y_pred4, index =[Atest.index]) 
final_df1=pred_df.add(pred_df2)
final_df2=final_df1.add(pred_df3)
final_df3=final_df2.add(pred_df4)
final=final_df3/4

In [ ]:
sub = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')
target_cols= targets.columns[1:207]
sub[target_cols] = 0
sub.loc[Atest.index,1:] = final.values
sub.loc[Atest.index,1:] =pred_df.clip(0.001,0.999)
sub.to_csv('submission.csv', index=False)

## <font color='blue'> REFERENCES </font>  
There are really many inspiring notebooks and discussions published for this contest covering various aspects, that helped me a lot in to build this kernel to mention a few:

* https://www.kaggle.com/c/lish-moa/discussion/184005
* https://www.kaggle.com/isaienkov/mechanisms-of-action-moa-prediction-eda
* https://www.kaggle.com/rahulsd91/moa-label-smoothing
* https://www.kaggle.com/nayuts/moa-pytorch-nn-pca-rankgauss

Thank you very much for your time reading this kernel. Please feel free to leave your comments and suggestions about how I can improve this work. And don't forget , if you found something that you liked or gave you an idea, do UPVOTE!
        
    